## Create Geometry

In [1]:
import numpy as np
import nibabel as nib
import itk
import json
import yaml

### Lung mask

In [2]:
atm255 = nib.load('/home/anna/Desktop/ATM_data/ATM_255/ATM_255_0000_seg_lobes.nii.gz')
atm255_data = atm255.get_fdata().astype(np.uint8)

In [3]:
for i in range(1, 6):
    lobe = np.where(atm255_data == i, 1, 0).astype(np.uint8)
    np.save(f'atm255_{i}.npy', lobe)
    itk.imwrite(itk.GetImageFromArray(lobe.astype(np.uint8)), f'atm255_{i}.nrrd')

## Bronchi roots

In [2]:
lobes = {}
# weird_spacing
x_min = -196.8
x_max = 202.4
y_min = -59.4
y_max = 339.8
z_min = 445.9
z_max = 843.9
p_min = np.array((x_min, y_min, z_min))
p_max = np.array((x_max, y_max, z_max))
size = np.array((512, 512, 797))

for lobe_idx in range(1, 6):
    with open(f'/home/anna/Desktop/ATM_data/ATM_255/lobe_markers/lobe{lobe_idx}.mrk.json') as f:
        lobe = json.load(f)
    points = lobe['markups'][0]['controlPoints']
    roots = []
    for i in range(len(points)//2):
        point1 = np.array(points[2*i]['position'])
        point2 = np.array(points[2*i+1]['position'])
        position = (point1-p_min) / (p_max-p_min) # normalizing to [0, 1]
        position *= size
        direction = (point2-point1) / np.linalg.norm(point2-point1)
        roots.append({
            'name': points[2*i]['label'][:3],
            'position': position.tolist(),
            'direction': direction.tolist()
        })
    lobes[f'lobe{lobe_idx}'] = roots
roots_dir = {
    'lobes': lobes
}
with open('/media/windows/Documents and Settings/Anna/Documents/UNI/23Masterthesis/OCTA-seg/vessel_graph_generation/geometries/roots.yml', 'w') as f:
    yaml.dump(lobes, f, default_flow_style=False, sort_keys=False)